In [2]:
import pandas as pd
from rapidfuzz import process, fuzz
import unicode
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from rapidfuzz import process, fuzz

# Google Sheets API 인증 및 접근
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("key.json", scope)
gc = gspread.authorize(creds)

# 스프레드시트 URL 설정
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PzFuHuF2DvMPdSI-TK4Kq5LjPwyGzIxH3Q0Drnqft20/edit?usp=sharing"

# URL을 통해 스프레드시트 열기
doc = gc.open_by_url(spreadsheet_url)

# 특정 시트 선택 (사본 페이지)
sheet = doc.worksheet("상병 Category의 사본")

# 시트의 모든 데이터를 가져오기 (2D 리스트 형태로 가져옴)
data = sheet.get_all_values()

# 데이터프레임으로 변환
df = pd.DataFrame(data[1:], columns=data[0])

eng_to_kor = {
    'r': 'ㄱ', 'R': 'ㄲ', 's': 'ㄴ', 'e': 'ㄷ', 'E': 'ㄸ', 'f': 'ㄹ', 'a': 'ㅁ', 'q': 'ㅂ', 'Q': 'ㅃ', 't': 'ㅅ',
    'T': 'ㅆ', 'd': 'ㅇ', 'w': 'ㅈ', 'W': 'ㅉ', 'c': 'ㅊ', 'z': 'ㅋ', 'x': 'ㅌ', 'v': 'ㅍ', 'g': 'ㅎ',
    'k': 'ㅏ', 'o': 'ㅐ', 'i': 'ㅑ', 'O': 'ㅒ', 'j': 'ㅓ', 'p': 'ㅔ', 'u': 'ㅕ', 'P': 'ㅖ', 'h': 'ㅗ', 
    'y': 'ㅛ', 'n': 'ㅜ', 'b': 'ㅠ', 'm': 'ㅡ', 'l': 'ㅣ'
}

# 초성 리스트
CHOSUNG_LIST = [
    'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ'
]

def get_chosung(text):
    result = []
    for char in str(text):
        if 44032 <= ord(char) <= 55203:
            index = (ord(char) - 44032) // 588
            result.append(CHOSUNG_LIST[index])
        else:
            result.append(char)
    return ''.join(result)

def convert_eng_to_kor(eng_text):
    return ''.join(eng_to_kor.get(char, char) for char in eng_text)

def is_chosung(text):
    return all(char in CHOSUNG_LIST for char in str(text))

def search_with_contains(data, query):
    return [item for item in data if query in item]

def search_with_partial_and_correction(data, query, threshold=60):
    matches = process.extract(query, data, scorer=fuzz.partial_ratio, limit=15)
    return [match[0] for match in matches if match[1] >= threshold]

def search_with_chosung_inclusion(data, query):
    chosung_query = get_chosung(query)
    return [item for item in data if chosung_query in get_chosung(item)]

def search_with_options(data, query):
    if query.startswith('K'):
        return search_with_contains(data, query[1:])

    if is_chosung(query):
        return search_with_chosung_inclusion(data, query)

    if all(char.isalpha() and char.islower() for char in query):
        query = unicode.join_jamos(convert_eng_to_kor(query))

    return search_with_partial_and_correction(data, query)

def search_df_with_options(df, query):
    results = search_with_contains(df.applymap(str).values.flatten(), query)
    if not results:
        for column in df.columns:
            col_results = search_with_options(df[column].apply(str).tolist(), query)
            if col_results:
                results.extend(col_results)

    unique_results = set(results)
    return df[df.apply(lambda row: any(str(row[col]) in unique_results for col in df.columns), axis=1)]


search_query = "ㄱㅂㅈㄹ"
result_df = search_df_with_options(df, search_query)

# 결과 출력
print("검색어:", search_query)
result_df

검색어: ㄱㅂㅈㄹ


pandas.core.frame.DataFrame